1. 商品的單價在 \[dbo\].\[ProductMeta\] 裡 MetaKey = Price，需從該表取得單價，再乘以銷售數量，更新到 \[dbo\].\[OrderProduct\] 的 \[ProductNetRevenue\] 銷售淨額欄位中
2. 每件商品的成本不同，因為無原始數據，因此為 ProductMeta 資料表，再新建一個 MetaKey = Cost ，MetaValue = 採 0.4~0.7 隨機的方式賦予
3. 根據 2 ，再計算 \[OrderProduct\] 表中的 \[ProductGrossRevenue\] 商品毛利
4. 依營業稅 5%的計算方式，計算商品稅額，寫入 \[OrderProduct\] 表中的 \[TaxAmount\] 欄位中
5. 依據 \[OrderProduct\] 內的資訊，寫入 \[OrderStats\] 表中的 \[TotalSales\], \[TaxTotal\], \[NetTotal\] 欄位

A ∪ B   Set union: 將兩個集合合二為一

A ∩ B   Set intersection: A 和 B 共有的成員

A − B   Set difference: 不在 B 中的 A 的成員

以上轉換成 SQL 語法:

A ∪ B : UNION or UNION ALL (UNION 消除重複, UNION ALL 全部保留)

A ∩ B : INTERSECT 相交

A − B : EXCEPT 除了

In [ ]:
-- 1. 先執行 Convert_json2sql_Products.ipynb 的練習1

-- 2. 因為有許多商品並沒有加入 price 這個欄位，所以要補齊
INSERT INTO [ProductMeta] ([ProductID] ,  [MetaKey],  [MetaValue]) 
SELECT [ID], 'price', '' FROM [Products]
EXCEPT
SELECT [ProductID], 'price', '' FROM [ProductMeta] WHERE [MetaKey] = 'price';

-- 3. 因為有許多都沒有 price ，所以再執行以下：(隨機100~200 之間的整數數字)
UPDATE ProductMeta SET [MetaValue] = FLOOR(RAND(CHECKSUM(NEWID()))*(200-100+1))+100 WHERE [MetaKey]= 'price' and [MetaValue] = '';

In [ ]:
-- 檢視 [Productmeta] 
SELECT TOP 1000 [ProductID], [MetaValue] FROM [ProductMeta] WHERE [MetaKey] = 'price' 

-- 發現其中有一筆資料的價格是 199-499 ，因為是文字屬性所以無法計算，只能暫且給予一個值，要解決這個問題需要從來源資料結構上著手
-- UPDATE ProductMeta SET MetaValue = 499 WHERE ProductID = 11 AND MetaKey = 'price' 

In [ ]:
-- 1.商品的單價在 [dbo].[ProductMeta] 裡 MetaKey = Price，需從該表取得單價，再乘以銷售數量，更新到 [dbo].[OrderProduct] 的 [ProductNetRevenue] 銷售淨額欄位中 --
UPDATE A 
SET A.[ProductNetRevenue] = B.[MetaValue] * A.[ProductQuantity]
FROM 
    [OrderProduct] AS A
    INNER JOIN [ProductMeta] AS B
    ON A.[ProductID] = B.[ProductID]
WHERE
    B.[MetaKey] = 'price';


1. `CAST()`: [https://docs.microsoft.com/en-us/sql/t-sql/functions/cast-and-convert-transact-sql](https://docs.microsoft.com/en-us/sql/t-sql/functions/cast-and-convert-transact-sql)
2. `RAND()`: [http://msdn.microsoft.com/en-us/library/ms177610.aspx](http://msdn.microsoft.com/en-us/library/ms177610.aspx)
3. `CHECKSUM()`: [http://msdn.microsoft.com/en-us/library/ms189788.aspx](http://msdn.microsoft.com/en-us/library/ms189788.aspx)
4. `NEWID()`: [https://docs.microsoft.com/en-us/sql/t-sql/functions/newid-transact-sql](https://docs.microsoft.com/en-us/sql/t-sql/functions/newid-transact-sql) 
5. 視覺化表達 SQL JOIN ： [https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins](https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins)

In [ ]:
-- 2.每件商品的成本不同，因為無原始數據，因此為 ProductMeta 資料表，再新建一個 MetaKey = Cost ，MetaValue = 採 40~70 % 隨機的方式賦予
-- Step 1
INSERT INTO [ProductMeta] ([ProductID] ,  [MetaKey],  [MetaValue]) 
SELECT [ProductID], 'cost', CAST(RAND(CHECKSUM(NEWID())) * 40 as INT) + 40 FROM [ProductMeta] WHERE [MetaKey] = 'price';

-- 因為 MetaValue 的欄位型別是 nvarchar ，沒辦法進行數學計算，所以要用 CAST 來轉換型別，以下是測試練習
-- SELECT CAST([MetaValue] AS DECIMAL(8,2) ) FROM ProductMeta WHERE [MetaKey] = 'cost'

-- Step 2
UPDATE A 
SET A.[MetaValue] = CAST(B.[MetaValue] AS DECIMAL(8,2) ) * CAST(A.[MetaValue] AS DECIMAL(8,2) ) / 100
FROM 
    [ProductMeta] AS A
    INNER JOIN [ProductMeta] AS B
    ON A.[ProductID] = B.[ProductID]
WHERE
    B.[MetaKey] = 'price' AND A.[MetaKey] = 'cost';

-- 問題：組合新的語法一次就新增完成？


In [ ]:
-- 3. 計算 [OrderProduct] 表中的 [ProductGrossRevenue] 商品毛利收入
-- SELECT A.[ProductID], A.[ProductNetRevenue] - (B.[MetaValue] * A.[ProductQuantity])
UPDATE A
SET A.[ProductGrossRevenue] = A.[ProductNetRevenue] - (B.[MetaValue] * A.[ProductQuantity])
FROM 
    [ProductMeta] AS B
    INNER JOIN [OrderProduct] AS A
    ON A.[ProductID] = B.[ProductID]
WHERE B.[MetaKey] = 'cost'

In [71]:
-- 4. 依營業稅 5% 的計算方式，計算商品稅額，寫入 [OrderProduct] 表中的 [TaxAmount] 欄位中(三聯發票的作法)
UPDATE [OrderProduct]
SET [TaxAmount] = [ProductNetRevenue] * 0.05

(90718 個資料列受到影響)

總執行時間: 00:00:00.248

In [73]:
-- 5. 依據 [OrderProduct] 內的資訊，寫入 [OrderStats] 表中的 [TotalSales], [TaxTotal], [NetTotal] 欄位
-- SELECT A.[ProductNetRevenue] + A.[TaxAmount], A.[TaxAmount], A.[ProductNetRevenue]
UPDATE B
SET B.TotalSales = A.[ProductNetRevenue] + A.[TaxAmount], B.TaxTotal = A.TaxAmount, B.NetTotal = A.ProductNetRevenue
FROM
    [OrderStats] AS B
    INNER JOIN [OrderProduct] AS A
    ON A.[OrderID] = B.[OrderID]


(90717 個資料列受到影響)

總執行時間: 00:00:02.233